In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras.models import Model
import savetonparray as savetonp
from google.colab import drive
from sklearn.utils import shuffle

# drive.mount('/content/drive')
# !rm -rf '/content/train'
# savetonp.run('test')
# train_list = os.listdir('/content/train')
# print(len(train_list))

In [ ]:
train_x = np.load('train.npy')
test_x = np.load('test.npy')
train_y = pd.read_csv('train_result.csv').iloc[:,1:].values
test_y = pd.read_csv('test_result.csv').iloc[:,1:].values

train_x = (train_x - np.min(train_x))/(np.max(train_x) - np.min(train_x))
test_x = (test_x - np.min(test_x))/(np.max(test_x) - np.min(test_x))
train_y = train_y/100.
test_y = test_y/100.

train_x, train_y = shuffle(train_x, train_y)
test_x, test_y = shuffle(test_x, test_y)
# shuffle train labels and samples with respect to each other

print ("train_x shape: " + str(train_x.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x shape: " + str(test_x.shape))
print ("test_y shape: " + str(test_y.shape))

train_x shape: (1005, 299, 299, 3)
train_y shape: (1005, 3)
test_x shape: (270, 299, 299, 3)
test_y shape: (270, 3)


In [ ]:
# build our classifier model based on pre-trained InceptionResNetV2:
# 1. we don't include the top (fully connected) layers of InceptionResNetV2
# 2. we add a DropOut layer followed by a Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')

model = InceptionResNetV2(include_top=False,
                        weights = 'imagenet',
                        input_tensor=None,
                        input_shape=(299,299,3))
x = model.output
x = Flatten()(x)
output_layer = Dense(3, activation='softmax', name='softmax')(x)
model_final = Model(inputs=model.input, outputs=output_layer)
for layer in model_final.layers[:2]:
    layer.trainable = False
for layer in model_final.layers[2:]:
    layer.trainable = True
model_final.compile(optimizer=Adam(lr=1e-5),
                  loss='MSE', metrics=['MAE'])

In [ ]:
print(model_final.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1421 (Conv2D)            (None, 149, 149, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1421 (Batch (None, 149, 149, 32) 96          conv2d_1421[0][0]                
__________________________________________________________________________________________________
activation_1421 (Activation)    (None, 149, 149, 32) 0           batch_normalization_1421[0][0]   
______________________________________________________________________________________________

In [ ]:
# train the model
model_final.fit(train_x, train_y, epochs = 2, batch_size = 15)


Epoch 1/2
67/67 [==============================] - 1550s 23s/step - loss: 0.0315 - MAE: 0.1380
Epoch 2/2
67/67 [==============================] - 1585s 24s/step - loss: 0.0162 - MAE: 0.0997


In [ ]:
# save trained weights
# model_final.save('3frames_15batch_3epochs_15batch_MSEMAE_normalizedinput_InceptionResnetV2.h5')
model_final.save_weights('3frames_15batch_3epochs_15batch_MSEMAE_normalizedinput_InceptionResnetV2.h5')

In [ ]:
preds = model_final.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
print(preds)

9/9 [==============================] - 90s 9s/step - loss: 0.0592 - MAE: 0.1604
Loss = 0.05922270566225052
Test Accuracy = 0.16038352251052856
[0.05922270566225052, 0.16038352251052856]


In [ ]:
y_pred_array = np.empty(((test_y.shape[0]), 3), dtype=int)
for i in range (test_x.shape[0]):
  x_pred = test_x[i,:,:,:]
  x_pred = x_pred.reshape((1,299,299,3))
  # print(x_pred.shape)
  y_pred = model_final.predict(x_pred)
  y_pred_array[i] = (y_pred*100).astype(int)

In [ ]:
print(y_pred_array)
print(test_y*100)
np.savetxt('predicted_y.csv', y_pred_array, fmt='%5.0f', delimiter=",")
np.savetxt('groundtruth_y.csv', test_y*100, fmt='%5.0f', delimiter=",")

[[53 38  8]
 [35 12 51]
 [43 34 21]
 [97  0  1]
 [24 63 11]
 [31  4 63]
 [43 20 35]
 [20 13 66]
 [40 18 40]
 [70 17 11]
 [58 15 25]
 [38 16 45]
 [33 19 47]
 [26  8 64]
 [21 64 14]
 [72 14 12]
 [39 36 24]
 [35 43 21]
 [46 34 18]
 [39 30 30]
 [39 44 15]
 [80  7 12]
 [96  1  2]
 [44 15 39]
 [26 46 27]
 [37 29 33]
 [39 34 25]
 [29 49 21]
 [40 30 28]
 [34 32 33]
 [47 25 26]
 [35 25 39]
 [59 23 17]
 [65  9 25]
 [37 25 37]
 [54 21 23]
 [56 28 14]
 [47 32 19]
 [67 13 19]
 [36 40 23]
 [36  9 54]
 [32 31 36]
 [88  8  3]
 [51 22 26]
 [18 37 44]
 [31 32 35]
 [41 31 27]
 [40 24 35]
 [36 33 30]
 [50 13 36]
 [36 43 20]
 [51 17 30]
 [41 34 24]
 [28 49 21]
 [57 22 20]
 [84  5 10]
 [45 36 17]
 [25 39 34]
 [37 44 17]
 [36 44 18]
 [58 18 22]
 [23 17 59]
 [56 25 17]
 [59 19 21]
 [28 59 12]
 [47 36 16]
 [18 66 14]
 [18 31 49]
 [27 16 55]
 [42 40 17]
 [12 35 52]
 [25 36 38]
 [43 16 39]
 [17 33 48]
 [73 20  5]
 [22 48 28]
 [97  0  1]
 [43 33 23]
 [67 26  5]
 [58 21 20]
 [93  2  3]
 [44 28 26]
 [67 18 13]
 [72

In [ ]:
test_y_val = test_y_val*100
np.savetxt('test_y_val.csv', test_y_val, fmt='%5.0f', delimiter=",")